In [ ]:
# pip install spacy
# python -m spacy download en_core_web_sm
# pip install --user -U nltk

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# From Example Code  https://github.com/Swathiu/Detecting-Fake-Reviews/blob/master/Deception_Detection.py
import pandas as pd
import numpy as np
import os

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re

from datetime import datetime
from time import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, pairwise_distances
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import spacy

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Read in Data

In [ ]:
overall_start_time = time()

file_path = "C:/Users/tsaie/OneDrive/Desktop/FARS (desktop local)/Datasets/"
category = 'Apparel' # Wireless, Tools, Software, Personal_Care_Appliances, Major_Appliances, Mobile_Apps, Mobile_Electronics, PC, Electronics, Automotive, Apparel, Beauty, Office_Products, Outdoors
file_name = f'amazon_reviews_us_{category}_v1_00.tsv.gz'

# Read in the specified number of rows
n_rows_to_read_in = 2_000_000
data = pd.read_csv(file_path + file_name, compression='gzip', header=0, sep='\t', quotechar='"', error_bad_lines=False, warn_bad_lines=False, nrows=n_rows_to_read_in)

# Read in ALL of the rows
# data = pd.read_csv(file_path + file_name, compression='gzip', header=0, sep='\t', quotechar='"', error_bad_lines=False, warn_bad_lines=False)

print(f"The 'data' file has {data.shape[0]} rows and {data.shape[1]} columns")
data.head(3)

In [ ]:
data_small = data[['verified_purchase', 'review_body']]
data_small.head()

## Create Balanced Dataset
- have same number of rows of verified and unverified reviews

In [ ]:
def under_sampling(df):
    print("Under-Sampling Data")
    # Count of Reviews
    print("Verified:   ", sum(df['verified_purchase'] == 'Y'))
    print("Un-Verified:", sum(df['verified_purchase'] == 'N'))

    sample_size = sum(df['verified_purchase'] == 'N')
    gold_reviews_df = df[df['verified_purchase'] == 'Y']
    fake_reviews_df = df[df['verified_purchase'] == 'N']

    gold_reviews_us_df = gold_reviews_df.sample(sample_size)
    under_sampled_df = pd.concat([gold_reviews_us_df, fake_reviews_df], axis=0)

    print("Under-Sampled Verified:   ", sum(under_sampled_df['verified_purchase'] == 'Y'))
    print("Under-Sampled Un-Verified:", sum(under_sampled_df['verified_purchase'] == 'N'))
    

    # Graph of Data Distribution
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.countplot(x='verified_purchase', data=under_sampled_df)
    plt.title("Count of Reviews")
    plt.show()
    print("Under-Sampling Complete")
    
    return under_sampled_df

In [ ]:
data_equal_weight = under_sampling(data_small)
# data_equal_weight

# Data Cleaning

In [ ]:
# Pre-processing Text Reviews
def data_cleaning(df):
    # Removing emtpy cells
    df.dropna(inplace=True)
    df['review_cleaned'] = df['review_body'].copy()
    
    # Removing Unicode Chars (URL)
    df['review_cleaned'] = df['review_cleaned'].apply(
        lambda rev: re.sub(r"(\w+:\/\/\S+)|^rt|http.+?", "", rev))
        
    # Replace HTML keywords with blank space ("&quot;", "br", "&#34")
    remove_dict = {"<br /><br />": " ", "<br />": " ", "br ": "", "&quot;": " ", "&#34": " ",
                   "<BR>": " ", "_": ""}
    for key, val in remove_dict.items():
        df['review_cleaned'] = df['review_cleaned'].apply(
            lambda x: x.replace(key, val))
        
    print("\n######## Remove URL and HTML Keywords Complete ########")
    
    # Remove Punctuations and numbers
    tokenizer = RegexpTokenizer(r'\w+')
    df['review_cleaned'] = df['review_cleaned'].apply(
        lambda x: ' '.join([word for word in tokenizer.tokenize(x)]))
    
    remove_dict = {"0": "", "1": "", "2": "", "3": "", "4": "", "5": "", "6": "", "7": "", "8": "", "9": "",
                   "(": "", ")":""}
    for key, val in remove_dict.items():
        df['review_cleaned'] = df['review_cleaned'].apply(
            lambda x: x.replace(key, val))
    
    print("\n######## Remove Punctuation and Numbers Complete ########")
    
    # Lowercase Words
    df['review_cleaned'] = df['review_cleaned'].str.lower()
    
    print("\n######## Lowercase Complete ########")

    # Remove Stop Words.
    stop = stopwords.words('english')
      
    df['review_cleaned'] = df['review_cleaned'].apply(
        lambda x: ' '.join([word for word in x.split() if word.strip() not in stop]))
    
    print("\n######## Remove Stop Words Complete ########")
    
    # Lemmatization using .lemma_
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    df['review_cleaned'] = df['review_cleaned'].apply(
        lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
    
    print("\n######## Data Cleaning Complete ########")
    
    return df

In [ ]:
# Clean the dataset
data_cleaned = data_cleaning(data_equal_weight)
data_cleaned.head()

# Feature Engineering + Prepare Data for Machine Learning

## Add Bigrams

In [ ]:
# https://stackoverflow.com/questions/48331315/how-to-extract-all-the-ngrams-from-a-text-dataframe-column-in-different-order-in

from collections import Counter
from nltk import ngrams
from itertools import chain

def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

def add_bigram_column(df):
    copy = df.copy()
    copy['bigrams'] = copy['review_cleaned'].map(lambda x: find_ngrams(x.split(), 2))
    return copy
    
data_cleaned = add_bigram_column(data_cleaned)
data_cleaned.head()

## Let's call the bigrams in vertified reviews "gold_bigrams" and the bigrams in unverified reviews "fake_bigrams"

In [ ]:
data_fake = data_cleaned[data_cleaned['verified_purchase'] == 'N']
data_gold = data_cleaned[data_cleaned['verified_purchase'] == 'Y']

data_fake.tail(1)

### Split the dataframe into data_for_bigram and data_for_ML

In [ ]:
df_fake = data_fake.sample(frac = 0.50)
data_for_ML_fake = data_fake.drop(df_fake.index)

df_gold = data_gold.sample(frac = 0.50)
data_for_ML_gold = data_gold.drop(df_gold.index)

data_cleaned = pd.concat([data_for_ML_fake, data_for_ML_gold], ignore_index=True)

print(f'df_fake has {len(df_fake)} rows.')
print(f'df_gold has {len(df_gold)} rows.')
print(f'data_cleaned has {len(data_cleaned)} rows.')

In [ ]:
def create_gold_bigrams(df_gold):
    global gold_bigrams, gold_bigram_counts
    gold_bigrams = df_gold['bigrams'].tolist()
    gold_bigrams = list(chain(*gold_bigrams))
    gold_bigram_counts = Counter(gold_bigrams)

create_gold_bigrams(df_gold)
print(f'gold_bigram_counts has {len(gold_bigram_counts)} items')
gold_bigram_counts.most_common(20)

In [ ]:
def create_fake_bigrams(df_fake):
    global fake_bigrams, fake_bigram_counts
    fake_bigrams = df_fake['bigrams'].tolist()
    fake_bigrams = list(chain(*fake_bigrams))
    fake_bigram_counts = Counter(fake_bigrams)
    
create_fake_bigrams(df_fake)
print(f'fake_bigram_counts has {len(fake_bigram_counts)} items')
fake_bigram_counts.most_common(20)

In [ ]:
bigram_file_path = "C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/Bigrams/"

gold_bigram_counts_df = pd.DataFrame(gold_bigram_counts.most_common(), columns=['bigram','count'])
gold_bigram_file_name = f'{category} gold_bigrams.csv'
gold_bigram_counts_df.to_csv(bigram_file_path + gold_bigram_file_name)

fake_bigram_counts_df = pd.DataFrame(fake_bigram_counts.most_common(), columns=['bigram','count'])
fake_bigram_file_name = f'{category} fake_bigrams.csv'
fake_bigram_counts_df.to_csv(bigram_file_path + fake_bigram_file_name)

## Create and Save Data Visualizations

In [ ]:
def plot_counter_N_most_common(gold_or_fake, N):
    counter = eval(f'{gold_or_fake}_bigram_counts')
    d = {}
    for tup in counter.most_common(N):
        bigram = f"{tup[0][0]} {tup[0][1]}"
        d[bigram] = tup[1]
    
    df = pd.Series(data=d).sort_values(ascending=False).to_frame().reset_index().rename(columns={'index': 'bigram', 0: 'count'})
    f, ax = plt.subplots(figsize=(8, N/4 + 1))
    if gold_or_fake == 'gold':
        palette = sns.color_palette('flare', n_colors=N)
    else:
        palette = sns.color_palette('crest', n_colors=N)
    palette.reverse()
    title = f'Top {N} Bigrams in {gold_or_fake.capitalize()} Reviews for the {category} Category'
    sns.barplot(x='count', y='bigram', data=df, orient='h', palette=palette).set_title(title)
    
    file_path = f"C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/Images/{category}/"
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    plt.savefig(file_path + f'{title}.png', bbox_inches='tight', transparent=True)

plot_counter_N_most_common("gold", 15)
plot_counter_N_most_common("fake", 15)
plot_counter_N_most_common("gold", 20)
plot_counter_N_most_common("fake", 20)
plot_counter_N_most_common("gold", 30)
plot_counter_N_most_common("fake", 30)

In [ ]:
from wordcloud import WordCloud

def wordcloud_N_most_common(gold_or_fake, N):
    counter = eval(f'{gold_or_fake}_bigram_counts')
    d = {}
    for tup in counter.most_common(N):
        bigram = f"{tup[0][0]} {tup[0][1]}"
        d[bigram] = tup[1]
       
    colors = 'summer' if gold_or_fake == 'fake' else 'magma'
    word_cloud = WordCloud(width=3000, height=1500, background_color=None, mode = "RGBA", colormap=colors)
    word_cloud.generate_from_frequencies(frequencies=d)

    plt.figure(figsize=(14,7))
    plt.tight_layout(pad=0)
    plt.imshow(word_cloud, interpolation='bilinear')
    plt.axis("off")
    title = f'WORDCLOUD - Most Popular Bigrams in {gold_or_fake.capitalize()} Reviews for the {category} Category'
    plt.title(title, fontsize=15)

    file_path = f"C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/Images/{category}/"    
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    plt.savefig(file_path + f'{title}.png', bbox_inches='tight', transparent=True)
   
    plt.show()
   
wordcloud_N_most_common('gold', 150)
wordcloud_N_most_common('fake', 150)

## Add Features

**count** = number of bigrams in a review

**percent** = number of gold/fake_bigrams as a percentage of total number of bigrams in a review.

**simple score** = sum of the gold/fake_bigrams' popularity scores (calculated using the bigram's count in the Counter)

**normalized score** = simple score / total bigram count

In [ ]:
def get_bigram_count(bigrams, bigram_dict):
    if len(bigrams) == 0:
        return 0
    
    count = 0
    for bigram in bigrams:
        if bigram in bigram_dict.keys():
            count += 1
    return count

def get_count_percent(bigrams, bigram_dict):
    if len(bigrams) == 0:
        return 0
    
    count = get_bigram_count(bigrams, bigram_dict)
    return count / len(bigrams)

def get_averge_top_score(bigrams, bigram_dict, topN=5):
    if len(bigrams) == 0:
        return 0
    
    scores = np.array([])
    for bigram in bigrams:
        if bigram in bigram_dict.keys():
            scores = np.append(scores, bigram_dict[bigram])
    
    if len(bigrams) <= topN:
        return scores.mean()
    
    sort_descending = -np.sort(-scores)[:topN]
    avg_top_score = sort_descending.mean()
    return avg_top_score

def get_normalized_score(bigrams, bigram_dict):
    if len(bigrams) == 0:
        return 0
    
    score = 0
    for bigram in bigrams:
        if bigram in bigram_dict.keys():
            score += bigram_dict[bigram]
    return score / len(bigrams)

def get_unique_percent(bigrams, bigram_dict, the_other_bigram_dict, unique_threshold=0):
    # Count the number of bigrams in a review that appear in bigram_dict but not the_other_bigram_dict. 
    # Can adjust unique_threshold so you can count also the bigrams that appear in ...
    # ...the_other_bigram_dict fewer than unique_threshold times
    
    if len(bigrams) == 0:
        return 0
    
    count = get_bigram_count(bigrams, bigram_dict)
    for bigram in bigrams:
        if bigram in the_other_bigram_dict.keys():
            if the_other_bigram_dict[bigram] > unique_threshold:
                count -= 1
    return count / len(bigrams)

def has_fake_keywords(bigrams):
    fake_keywords = ['honest', 'unbiased', 'unbias', 'biased', 'bias', 'neutral', 'impartial', 'truthful', 
                     'discount', 'free', 'promotion', 'promote', 'complimentary', 'test', 'influence', 'influencer',
                     'independent']
    fake_tuples = [('receive', 'product'), ('product', 'receive'), ('provide', 'review'), 
                   ('product', 'exchange'), ('exchange', 'review'), ('review', 'opinion'),
                   ('sample', 'provide'), ('provide', 'sample'), ('sample', 'review'), ('review', 'sample'), 
                   ('sample', 'product'), ('supply', 'sample'), ('receive', 'sample'), ('sample', 'receive')] 
    
    for kw in fake_tuples:
        if kw in bigrams:
            return True
            
    for bigram in bigrams:
        for kw in fake_keywords:
            if kw in bigram:
                return True
    
    return False



# Add bigram_count
data_cleaned['bigram_count'] = data_cleaned['bigrams'].apply(lambda x: len(x))

# Add features based on gold or fake bigrams
fake_bigram_dict = dict(fake_bigram_counts)
fake_bigram_dict_filtered = dict((k, v) for k, v in fake_bigram_dict.items() if v >= 2)

gold_bigram_dict = dict(gold_bigram_counts)
gold_bigram_dict_filtered = dict((k, v) for k, v in gold_bigram_dict.items() if v >= 2)


def add_gold_fake_features(df):
    
    for gold_or_fake in ['gold', 'fake']:

        exec(f"df['{gold_or_fake}%'] = df['bigrams'].apply(\
            lambda x: get_count_percent(x, {gold_or_fake}_bigram_dict_filtered))")
        
        exec(f"df['{gold_or_fake}_unique%'] = df['bigrams'].apply(\
            lambda x: get_unique_percent(x, {gold_or_fake}_bigram_dict, {gold_or_fake}_bigram_dict, 3))")

        exec(f"df['{gold_or_fake}_score'] = df['bigrams'].apply(\
            lambda x: get_normalized_score(x, {gold_or_fake}_bigram_dict))")
        
        exec(f"df['{gold_or_fake}_top_score'] = df['bigrams'].apply(\
            lambda x: get_averge_top_score(x, {gold_or_fake}_bigram_dict, 1))")
        
        exec(f"df['{gold_or_fake}_top_avg_score'] = df['bigrams'].apply(\
            lambda x: get_averge_top_score(x, {gold_or_fake}_bigram_dict, 5))")

    df['has_fake_keywords'] = df['bigrams'].apply(lambda x: has_fake_keywords(x))
    
    return df

data_cleaned = add_gold_fake_features(data_cleaned).fillna(0)

data_cleaned.tail(1)


# Use Machine Learning to Make Predictions for Verified VS. Unverified
LABELS: 
- 1 = verified review
- 0 = unverified review

In [ ]:
features = ['bigram_count', 
            'fake%', 'fake_unique%', 'fake_score', 'fake_top_avg_score', 'fake_top_score',
            'gold%', 'gold_unique%', 'gold_score', 'gold_top_avg_score', 'gold_top_score',
            'has_fake_keywords']

'''
['bigram_count', 
            'fake%', 'fake_unique%', 'fake_score', 'fake_top_avg_score', 'fake_top_score',
            'gold%', 'gold_unique%', 'gold_score', 'gold_top_avg_score', 'gold_top_score',
            'has_fake_keywords']
'''

def semi_supervised_learning(df, model, algorithm, threshold=0.8, iterations=40):
    df = df.copy()
    
    df_unlabled = df[features]


    df['verified_purchase'] = df['verified_purchase'].apply(lambda x: 1 if x == 'Y' else 0)
    print("Training " + algorithm + " Model")
    labels = df['verified_purchase']
    
    train_data, test_data, train_label, test_label = train_test_split(df_unlabled, labels, test_size=0.25, random_state=42)

    test_data_copy = test_data.copy()
    test_label_copy = test_label.copy()
    
    all_labeled = False

    current_iteration = 0

    pbar = tqdm(total=iterations)

    while not all_labeled and (current_iteration < iterations):
        current_iteration += 1
        model.fit(train_data, train_label)

        probabilities = model.predict_proba(test_data)
        pseudo_labels = model.predict(test_data)

        indices = np.argwhere(probabilities > threshold)

        for item in indices:
            train_data.loc[test_data.index[item[0]]] = test_data.iloc[item[0]]
            train_label.loc[test_data.index[item[0]]] = pseudo_labels[item[0]]
        test_data.drop(test_data.index[indices[:, 0]], inplace=True)
        test_label.drop(test_label.index[indices[:, 0]], inplace=True)

        print("--" * 20)

        if len(test_data) == 0:
            print("Exiting loop")
            all_labeled = True
        pbar.update(1)
        
    pbar.close()
    predicted_labels = model.predict(test_data_copy)
    
    # RECORD TEST RESULTS
    test_df = test_data_copy.copy()
    test_df['predicted_label'] = predicted_labels
    test_df['true_label'] = list(test_label_copy)
    
    
    print(algorithm + ' Model Results')
    print('--' * 20)
    accuracy = accuracy_score(test_label_copy, predicted_labels)
    print('Accuracy Score : ' + str(accuracy))
    print('Precision Score : ' + str(precision_score(test_label_copy, predicted_labels, pos_label=1)))
    print('Recall Score : ' + str(recall_score(test_label_copy, predicted_labels, pos_label=1)))
    print('F1 Score : ' + str(f1_score(test_label_copy, predicted_labels, pos_label=1)))
#     print('Confusion Matrix : \n' + str(confusion_matrix(test_label_copy, predicted_labels)))
    plot_confusion_matrix(test_label_copy, predicted_labels, classes=[0, 1],
                          title=algorithm + ' Confusion Matrix').show()
    
    return model, test_df, round(accuracy, 4)


def plot_confusion_matrix(y_true, y_pred, classes, title=None, cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes,
           yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    return plt

## ML Method #1: Random Forest Classifier

In [ ]:
start_time = time()
# rf = RandomForestClassifier(random_state=42, criterion='entropy', max_depth=14, max_features='auto', n_estimators=500)
rf = RandomForestClassifier(random_state=42, criterion='entropy', max_depth=16, max_features='auto', n_estimators=500)
rf_model, test_df, accuracy = semi_supervised_learning(data_cleaned, model=rf, threshold=0.7, iterations=10, algorithm='Random Forest')
end_time = time()

print("Time taken : ", end_time - start_time)

## Result Analysis

In [ ]:
true_0 = test_df[test_df['true_label'] == 0]
true_1 = test_df[test_df['true_label'] == 1]

In [ ]:
def get_stats(df):
    gold_stat = round(df['gold_unique%'].mean(), 3)
    print(f"gold_stat {gold_stat}")
    
    fake_stat = round(df['fake_unique%'].median(), 3)
    print(f"fake_stat {fake_stat}")
    
print('true_1 (gold)\n')
get_stats(true_1)

print('\ntrue_0 (fake)\n')
get_stats(true_0)

In [ ]:
cond = test_df[(test_df['has_fake_keywords'])]
size = cond.shape[0]
stat_1 = round(cond['true_label'].mean(), 3)
stat_0 = round(1 - stat_1, 3)
print(f"gold percentage = {stat_1}  ({round(stat_1 * size)} / {size})")
print(f"fake percentage = {stat_0}  ({round(stat_0 * size)} / {size})")

In [ ]:
x = 'gold_score'
y = 'fake_score'
x1, y1 = true_1[x], true_1[y]
x2, y2 = true_0[x], true_0[y]
fig = plt.figure(figsize=(18, 10))
ax1 = fig.add_subplot(111)

ax1.scatter(x2, y2, s=3, alpha=0.5, c='r', marker="o", label='fake')
ax1.scatter(x1, y1, s=3, alpha=0.5, c='b', marker="o", label='gold')
# ax1.scatter(x2, y2, s=3, alpha=0.2, c='r', marker="o", label='fake')
plt.xlabel(x, fontsize=15)
plt.ylabel(y, fontsize=15)
plt.legend(loc='upper left', fontsize=15);
plt.show()

In [ ]:
test_result = test_df[['true_label', 'predicted_label']].merge(data_cleaned, how='left', left_index=True, right_index=True)
ML_test_result_file_path = "C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/ML Test Result (CSV)/"
ML_test_result_file_name = f'2022_05_11 {(category)} Test Result.csv'
test_result.to_csv(ML_test_result_file_path + ML_test_result_file_name)

In [ ]:
test_result[(test_result['true_label'] == 0) & (test_result['predicted_label'] == 1)].sample(3).drop(columns=['predicted_label','true_label'])

In [ ]:
z = test_result[(test_result['has_fake_keywords'])]
z[z['true_label'] == 0].sample(1)

In [ ]:
keyword = 'honest'
N = min(len(gold_bigram_counts), len(fake_bigram_counts))
print(f'fake | {sum([keyword in tup for tup in dict(fake_bigram_counts.most_common(N)).keys()])}')
print(f'gold | {sum([keyword in tup for tup in dict(gold_bigram_counts.most_common(N)).keys()])}')

In [ ]:
from joblib import dump, load
ML_model_file_path = "C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/ML Models/"
ML_model_file_name = f'2022_05_11 rf_model {(category)}.joblib'
dump(rf_model, ML_model_file_path + ML_model_file_name, compress=9)  

In [ ]:
overall_end_time = time()
print(round(overall_end_time - overall_start_time), "seconds")

# NEXT STEP: Making Predictions for a User Input

In [ ]:
# ML_model_file_path = "C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/ML Models/"
# ML_model_file_name = f'2022_05_05 rf_model {(category)}.joblib'
rf_model = load(ML_model_file_path + ML_model_file_name) 

bigram_file_path = "C:/Users/tsaie/OneDrive/Desktop/FARS/Ester Tsai (Bigram and ML)/Bigrams/"

gold_bigram_file_name = f'{(category)} gold_bigrams.csv'
gold_bigram_df = pd.read_csv(bigram_file_path + gold_bigram_file_name, index_col=0)
display(gold_bigram_df.head(3))

fake_bigram_file_name = f'{(category)} fake_bigrams.csv'
fake_bigram_df = pd.read_csv(bigram_file_path + fake_bigram_file_name, index_col=0)
display(fake_bigram_df.head(3))

In [ ]:
def gold_bigram_df_to_vars(gold_bigram_df):
    global gold_bigrams, gold_bigram_dict, gold_bigram_dict_filtered
    gold_bigrams = gold_bigram_df['bigram'].to_list()
    str_to_tuple = lambda x: (x.split("'")[1], x.split("'")[3])
    gold_bigrams = list(map(str_to_tuple, gold_bigrams))
    gold_bigram_dict = {gold_bigrams[i]: gold_bigram_df['count'].iloc[i] for i in range(len(gold_bigrams))}
    gold_bigram_dict_filtered = dict((k, v) for k, v in gold_bigram_dict.items() if v >= 2)
    
def fake_bigram_df_to_vars(fake_bigram_df):
    global fake_bigrams, fake_bigram_dict, fake_bigram_dict_filtered
    fake_bigrams = fake_bigram_df['bigram'].to_list()
    str_to_tuple = lambda x: (x.split("'")[1], x.split("'")[3])
    fake_bigrams = list(map(str_to_tuple, fake_bigrams))
    fake_bigram_dict = {fake_bigrams[i]: fake_bigram_df['count'].iloc[i] for i in range(len(fake_bigrams))}
    fake_bigram_dict_filtered = dict((k, v) for k, v in fake_bigram_dict.items() if v >= 2)
    
    
    
    
def clean_review_and_add_features(review, gold_bigram_df, fake_bigram_df):
    # Create dataframe for the singular review
    df = pd.DataFrame(data={'review_body': review}, index=[0])
    
    # Generate gold/{gold_or_fake} bigrams in order to score the user input
    gold_bigram_df_to_vars(gold_bigram_df)
    fake_bigram_df_to_vars(fake_bigram_df)
    
    # Clean the user input
    df = data_cleaning(df) 
#     df = add_bigram_column(df)
    df['bigrams'] = df['review_cleaned'].map(lambda x: find_ngrams(x.split(), 2))
    
    # Add features
    df['bigram_count'] = df['bigrams'].apply(lambda x: len(x))
    
    df = add_gold_fake_features(df)

    return df

## Try changing the review and see what the RandomForest model predicts!

In [ ]:
review = "Turbo charger, small and portable and fits perfectly in your pocket! A nice boost over regular chargers. I received this in exchange for an unbiased review."
#9994 "This is a thick high quality cable, my picture is crystal clear and this is just the right length for my needs, I received this in exchange for my honest review."
#112 Small,lightweight,fits on my desk. Does not come with batteries,I already had some.I received a discount for this review.
#59356 Turbo charger, small and portable and fits perfectly in your pocket! A nice boost over regular chargers. I received this in exchange for an unbiased review.

start_time = time()

# Clean the user input and create a dataframe for it
user_input_processed_df = clean_review_and_add_features(review, gold_bigram_df, fake_bigram_df)
display(user_input_processed_df)

def prepare_df_for_prediction(processed_df):

    return processed_df[features]

df_for_prediction = prepare_df_for_prediction(user_input_processed_df)

prediction, probabilities = rf_model.predict(df_for_prediction), rf_model.predict_proba(df_for_prediction)[0]

def interpret_prediction(review, pred, proba):
    prob_unverified, prob_verified = round(proba[0] * 100, 1), round(proba[1] * 100, 1)
    print(f'\nREVIEW: "{review}" \n')
    if pred == 1:
        print(f'PREDICTION: VERIFIED ({prob_verified}%) | UNVERIFIED ({prob_unverified}%)')
    if pred == 0:
        print(f'PREDICTION: UNVERIFIED ({prob_unverified}%) | VERIFIED ({prob_verified}%)')

end_time = time()
# print(round(end_time - start_time), "seconds")

interpret_prediction(review, prediction[0], probabilities)

# END